In [1]:
import os
import math
from dataclasses import dataclass
from typing import Iterable, Dict, Optional

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# ==== Add these helpers once ====

# ----------------------------
# Repro/Device/Small Defaults
# ----------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

LSTM_SEQ_LEN = 3
LSTM_HIDDEN  = 16
LSTM_LAYERS  = 1
LSTM_EPOCHS  = 15
LSTM_BATCH   = 64
LSTM_LR      = 1e-2
LSTM_DROPOUT = 0.0
MIN_SEG = 20

import numpy as np, math, torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMRegressor(nn.Module):
    def __init__(self, input_size=1, hidden=64, layers=1, dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden, num_layers=layers, batch_first=True,
                            dropout=dropout if layers > 1 else 0.0)
        self.fc = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)           # [B,L,H]
        yhat = self.fc(out[:, -1, :])   # [B,1]
        return yhat.squeeze(-1)

def build_sequences(y_window: np.ndarray, seq_len: int, start_abs_idx: int):
    """
    Build sequences for a contiguous window y_window (1D float32).
    Returns X_all [N,L,1], y_all [N], t_abs [N] with absolute target indices.
    """
    L = seq_len
    n = len(y_window)
    if n <= L: return None, None, None
    X = np.lib.stride_tricks.sliding_window_view(y_window, L+1)  # [N, L+1]
    X, y = X[:, :-1], X[:, -1]                                   # [N,L], [N]
    t_abs = np.arange(start_abs_idx + L, start_abs_idx + n, dtype=np.int64)  # absolute t of targets
    X = X[..., None].astype(np.float32)                          # [N,L,1]
    y = y.astype(np.float32)
    return X, y, t_abs

def fit_eval_sse(X, y, *, epochs=20, batch=128, lr=1e-3, hidden=64, layers=1, dropout=0.0, seed=0):
    """
    Train one LSTM on (X,y) and return (SSE, MSE, m, yhat, resid).
    In-sample evaluation (as in classical LR/QLR).
    """
    if X is None or len(y) == 0: return math.inf, math.inf, 0, None, None
    # torch.manual_seed(seed)
    if DEVICE.type == "cuda":
        # torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True

    ds = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    dl = DataLoader(ds, batch_size=batch, shuffle=True,
                    generator=torch.Generator(device="cpu"), #.manual_seed(seed),
                    pin_memory=(DEVICE.type=="cuda"), num_workers=0)

    model = LSTMRegressor(hidden=hidden, layers=layers, dropout=dropout).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for _ in range(epochs):
        model.train()
        for xb, yb in dl:
            xb = xb.to(DEVICE); yb = yb.to(DEVICE)
            opt.zero_grad()
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()

    # in-sample SSE
    model.eval()
    with torch.no_grad():
        all_pred, all_y = [], []
        evl = DataLoader(ds, batch_size=512, shuffle=False, pin_memory=(DEVICE.type=="cuda"))
        for xb, yb in evl:
            xb = xb.to(DEVICE); yb = yb.to(DEVICE)
            pred = model(xb)
            all_pred.append(pred.cpu().numpy()); all_y.append(yb.cpu().numpy())
        yhat = np.concatenate(all_pred); y_true = np.concatenate(all_y)
    resid = y_true - yhat
    SSE = float(np.sum(resid**2))
    m   = int(y_true.shape[0])
    MSE = SSE / max(m, 1)
    return SSE, MSE, m, yhat.astype(np.float32), resid.astype(np.float32)

def draw_rademacher(m, rng):
    return (rng.integers(0, 2, size=m) * 2 - 1).astype(np.float32)

def draw_mammen(m, rng):
    p = (np.sqrt(5)+1)/(2*np.sqrt(5))
    a = (1 - np.sqrt(5))/2   # ≈ -0.618
    b = (1 + np.sqrt(5))/2   # ≈  1.618
    u = rng.random(m)
    w = np.where(u < p, a, b).astype(np.float32)
    return w


In [3]:
def Likelihood_LSTM(SSE, T):
    return -(T / 2) * np.log(SSE)

In [9]:
import time
# SEED=123
# ----------------------------
# Detection run (with weighted bootstrap retraining)
# ----------------------------
def detect_changes_with_lstm(Data_N,
                             seq_len=LSTM_SEQ_LEN,
                             n_0=200,
                             jump=10,
                             search_step=5,
                             c=1.35,            # currently unused but kept for parity with your interface
                             alpha=0.95,
                             num_bootstrap=50,
                             epochs=LSTM_EPOCHS,
                             val_frac=0.2):
    """
    Data_N: 1D numpy array (time series)
    Returns (DataFrame, tests) with diagnostics per step.
    Uses OOS MSE and **per-batch weighted retraining** in bootstrap if enabled.
    """
    DT_N = pd.DataFrame({"Date": np.arange(len(Data_N)), "N": Data_N})
    windows, mse_vals, rmse_vals, likelihoods, scaled_windows = [], [], [], [], []
    tests = 0

    io = Data_N.shape[0] #starting from final time
    I_0 = list(Data_N[max(0,io-n_0):io])

    n_k_minus1 = n_0 
    I_k_minus1 = I_0

    for l in range(0, Data_N.shape[0], jump):
        try:
            t0 = time.time()
            io = Data_N.shape[0] - l

            # arithmetic schedule
            K = int(io / n_0)

            #geometric
            # K = max(0, math.ceil((math.log(io) - math.log(n_0)) / math.log(c)))

            I_0 = list(Data_N[max(0,io-n_0):io])
            I_k_minus1 = I_0
            n_k_minus1 = n_0

            for k in range(1, K + 1):
                # print(f"K={k}")
                # print(n_k_minus1)
                try:
                    #arithmetic
                    n_k       = (k + 1) * n_0
                    n_k_plus1 = (k + 2) * n_0

                    #geometric
                    # n_k = int(n_0 * c**k)
                    # n_k_plus1 = int(n_0 * c**(k + 1))

                    I_k = list(Data_N[max(0,io-n_k):io])
                    I_k_plus1 = list(Data_N[max(0,io-n_k_plus1):io])

                    # Pooled (observed) on I_k_plus1, out-of-sample
                    # === Precompute all sequences for the CURRENT window I_k_plus1 once ===
                    y_win = np.asarray(I_k_plus1, dtype=np.float32)
                    start_abs = max(0, io - n_k_plus1)
                    X_all, y_all, t_abs = build_sequences(y_win, seq_len, start_abs)
                    if X_all is None:
                        print("Window too small for sequences"); continue
                    
                    # --- Global null fit on I_{k+1} (observed) ---
                    SSE_I, MSE_I, m_I, yhat_I, resid_I = fit_eval_sse(
                        X_all, y_all, epochs=epochs, batch=LSTM_BATCH, lr=LSTM_LR,
                        hidden=LSTM_HIDDEN, layers=LSTM_LAYERS#, seed=SEED
                    )

                    # Candidate split range
                    J_start = max(seq_len, io - n_k)   # have enough left history
                    J_end   = io - n_k_minus1          # ensure right side at least n_0
                    if J_end <= J_start:
                        print(f"J_end ({J_end}) < J_start ({J_start})")
                        continue

                    
                    # Candidate split range in ABSOLUTE target indices
                    J_abs = np.arange(J_start, J_end, search_step, dtype=np.int64)
                    
                    T_vals = []
                    boot_T_by_split = []   # will store bootstrap Sup over splits per replication (for efficiency)
                    
                    # --- Observed T(i) across splits ---
                    for i_abs in J_abs:
                        # Strict no-leak masks by target index
                        Lmask = t_abs <= i_abs
                        Rmask = t_abs >  i_abs
                        mA = int(np.sum(Lmask)); mB = int(np.sum(Rmask))
                        if mA < 20 or mB < 20:      # min targets per side; tune as needed
                            T_vals.append(0.0); continue
                    
                        SSE_A, _, m_A, _, _ = fit_eval_sse(X_all[Lmask], y_all[Lmask],
                                                         epochs=epochs, batch=LSTM_BATCH, lr=LSTM_LR,
                                                         hidden=LSTM_HIDDEN, layers=LSTM_LAYERS, #seed=SEED
                                                        )
                        SSE_B, _, m_B, _, _ = fit_eval_sse(X_all[Rmask], y_all[Rmask],
                                                         epochs=epochs, batch=LSTM_BATCH, lr=LSTM_LR,
                                                         hidden=LSTM_HIDDEN, layers=LSTM_LAYERS, #seed=SEED
                                                        )

                        Ti = Likelihood_LSTM(SSE_A, m_A) + Likelihood_LSTM(SSE_B, m_B) - Likelihood_LSTM(SSE_I, m_I)
                        T_vals.append( max(0.0, Ti) )
                    
                    # --- Bootstrap critical values via wild residual bootstrap under the null ---
                    if num_bootstrap <= 0:
                        raise ValueError(f"Num bootstrap must be at least 1. {num_bootstrap} provided")
                    
                    rng = np.random.default_rng() #SEED
                    Sup_boot = np.empty(num_bootstrap, dtype=np.float64)  # store sup over splits for each b
                    
                    for b in range(num_bootstrap):
                        # multipliers (choose Mammen or Rademacher)
                        w = draw_mammen(len(resid_I), rng)   # or draw_rademacher(...)
                        y_star = (yhat_I + w * resid_I).astype(np.float32)
                    
                        # Refit global null on y* to get SSE_I*
                        SSE_Ib, _, m_Ib, yhat_Ib, resid_Ib = fit_eval_sse(
                            X_all, y_star, epochs=epochs, batch=LSTM_BATCH, lr=LSTM_LR,
                            hidden=LSTM_HIDDEN, layers=LSTM_LAYERS #, seed=SEED + b + 1
                        )
                    
                        # Sweep splits and take sup
                        sup_b = 0.0
                        for i_abs in J_abs:
                            Lmask = t_abs <= i_abs
                            Rmask = t_abs >  i_abs
                            mA = int(np.sum(Lmask)); mB = int(np.sum(Rmask))
                            if mA < 20 or mB < 20:    # same min-seg rule
                                continue
                            SSE_A_b, _, m_Ab, _, _ = fit_eval_sse(X_all[Lmask], y_star[Lmask],
                                                               epochs=epochs, batch=LSTM_BATCH, lr=LSTM_LR,
                                                               hidden=LSTM_HIDDEN, layers=LSTM_LAYERS #, seed=SEED + b + 11
                                                              )
                            SSE_B_b, _, m_Bb, _, _ = fit_eval_sse(X_all[Rmask], y_star[Rmask],
                                                               epochs=epochs, batch=LSTM_BATCH, lr=LSTM_LR,
                                                               hidden=LSTM_HIDDEN, layers=LSTM_LAYERS #, seed=SEED + b + 21
                                                              )
                            
                            Ti_b = Likelihood_LSTM(SSE_A_b, m_Ab) + Likelihood_LSTM(SSE_B_b, m_Bb) - Likelihood_LSTM(SSE_Ib, m_Ib)
                            
                            if Ti_b > sup_b: sup_b = Ti_b
                        Sup_boot[b] = max(0.0, sup_b)
                    
                    # --- Decision for the current window ---
                    if len(T_vals) > 0:
                        test_value = float(np.max(T_vals))
                        critical_value = float(np.quantile(Sup_boot, alpha))
                        print(f"[QLR] step={l} |I_k+1=[{max(0,io-n_k_plus1)}, {io}] | I_k=[{max(0,io-n_k)}, {io}]  |"
                              f"I_k-1=[{max(0,io-n_k_minus1)}, {io}] | J_k=[{J_start}, {J_end}] | k={k} | "
                              f"SupLR={test_value:.3f} | crit({alpha:.2f})={critical_value:.3f} | #splits={len(T_vals)} | B={num_bootstrap}")
                    else:
                        test_value, critical_value = 0.0, math.inf
                    
                    if test_value > critical_value:
                        print(f"Found break at step {l} (window size {len(I_k)}).")
                        break
                    else:
                        I_k_minus1 = I_k
                        n_k_minus1 = n_k
                        continue


                except ValueError as e:
                    print(f"[DEBUG] {e} | likely too-short segment")
                    continue
            
            # Record diagnostics
            if K == 0:
                I_k = I_0

            # For the per-step diagnostic, compute OOS MSE on I_k
            MSE_I_k = 0#segment_oos_mse(I_k, seq_len=seq_len, epochs=epochs, error_type=error_type)
            RMSE_I_k = 0#math.sqrt(max(MSE_I_k, 0.0))
            Likelihood_I_k = 0

            windows.append(len(I_k))
            mse_vals.append(MSE_I_k)
            rmse_vals.append(RMSE_I_k)
            likelihoods.append(Likelihood_I_k)
            
            scaled_windows.append(len(I_k) / io)

            t1 = time.time()
            print(f"Step {l:4d} | time/step={t1 - t0:.2f}s | window={len(I_k)} | RMSE={RMSE_I_k:.4f}")

        except ValueError as e:
            print(f"[DEBUG] {e} | skipping step {l}")
            windows.append(np.nan); mse_vals.append(np.nan); rmse_vals.append(np.nan); scaled_windows.append(np.nan)
            continue

    # Reverse to align like your original flow
    windows.reverse(); mse_vals.reverse(); rmse_vals.reverse(); scaled_windows.reverse()
    DT_N["windows_1"]        = pd.Series(windows)
    DT_N["scaled_windows_1"] = pd.Series(scaled_windows)
    DT_N["MSE_lstm_1"]       = pd.Series(mse_vals)
    DT_N["RMSE_lstm_1"]      = pd.Series(rmse_vals)

    return DT_N, tests

In [10]:
# Data. Generated with: https://github.com/QuantLet/AR_sim_p/tree/main
df = pd.read_csv("LPA/Simulation/data.csv")

In [ ]:
# ----------------------------
# Example: repeat experiment num_runs times
# ----------------------------
if __name__ == "__main__":
    # ======= experiment config =======
    num_runs  = 10     # total repetitions of the whole experiment
    boot_B    = 10      
    alpha     = 0.95
    n_0       = 100
    search_step = 10 #int(n_0 / 5)
    epochs    = 50
    jump=50
    # =================================

    dir_path = f"LPA/Simulation/Jump_{jump}_N0_{n_0}"
    try:
        os.makedirs(dir_path, exist_ok=True)
    except OSError as e:
        print(f"Error: {e}")


    n_total = 1500
    coeffs = [
        (0.9,  0.01, 0.01),
        (0.01, 0.9,  0.01),
        (0.01, 0.01, 0.9),
    ]
    change_points = [0, 500, 1000]

    for run in range(num_runs):
        print(f"\n=== RUN {run+1}/{num_runs} ===")

        Data_N = df["N"].to_numpy(dtype=np.float32)

        DT_out, total_tests = detect_changes_with_lstm(
            Data_N,
            seq_len=LSTM_SEQ_LEN,
            n_0=n_0,
            jump=jump,
            search_step=search_step,
            c=1.35,
            alpha=alpha,
            num_bootstrap=boot_B,
            epochs=LSTM_EPOCHS,
            val_frac=0.2
        )
        out_name = f"{dir_path}/LSTM_{n_total}_run{run:03d}_n{n_0}_alpha{alpha}_bootB{boot_B}.csv"
        DT_out.to_csv(out_name, index=False)
        print(f"Saved results to: {out_name}")


=== RUN 1/10 ===
[QLR] step=0 |I_k+1=[1200, 1500] | I_k=[1300, 1500]  |I_k-1=[1400, 1500] | J_k=[1300, 1400] | k=1 | SupLR=90.706 | crit(0.95)=84.310 | #splits=10 | B=10
Found break at step 0 (window size 200).
Step    0 | time/step=5.07s | window=200 | RMSE=0.0000
[QLR] step=50 |I_k+1=[1150, 1450] | I_k=[1250, 1450]  |I_k-1=[1350, 1450] | J_k=[1250, 1350] | k=1 | SupLR=85.589 | crit(0.95)=93.273 | #splits=10 | B=10
[QLR] step=50 |I_k+1=[1050, 1450] | I_k=[1150, 1450]  |I_k-1=[1250, 1450] | J_k=[1150, 1250] | k=2 | SupLR=119.165 | crit(0.95)=131.054 | #splits=10 | B=10
[QLR] step=50 |I_k+1=[950, 1450] | I_k=[1050, 1450]  |I_k-1=[1150, 1450] | J_k=[1050, 1150] | k=3 | SupLR=221.479 | crit(0.95)=196.624 | #splits=10 | B=10
Found break at step 50 (window size 400).
Step   50 | time/step=19.29s | window=400 | RMSE=0.0000
[QLR] step=100 |I_k+1=[1100, 1400] | I_k=[1200, 1400]  |I_k-1=[1300, 1400] | J_k=[1200, 1300] | k=1 | SupLR=76.155 | crit(0.95)=93.132 | #splits=10 | B=10
[QLR] step=100 